In [11]:
import os
from pathlib import Path

if not 'workbook_dir' in globals():
    workbook_dir = os.getcwd()
print('workbook_dir: ' + workbook_dir)
os.chdir(Path(workbook_dir).parent.absolute())
print("current working dir:", os.getcwd())

workbook_dir: /Users/minor/projects/mcs-formalizations/queries
current working dir: /Users/minor/projects/mcs-formalizations


In [12]:
from rdflib import Graph, URIRef
from mcs_formalizations.namespace import bind_namespaces, MCS, RDF
from mcs_formalizations.helpers import uri_base

### Question: True/False - A parking garage is larger than an automobile.

##### Prepare the Graph

In [13]:
parking_garage = uri_base("parking_garage")
automobile = uri_base("automobile")
vehicle = uri_base("vehicle")

##### Minimum Knowledge Assumed:
- a parking garage contains vehicles
- an automobile is a type of vehicle
- a container is larger than the things it contains

In [14]:
g = Graph()
bind_namespaces(g)

g.add((parking_garage, MCS.contains, vehicle))
g.add((vehicle, RDF.type, automobile))


In [15]:
with open("constructs/is_larger_than.txt", "r") as construct_file:
    isLargerThan = construct_file.read()

##### Execute the Construct

In [16]:
qres_construct = g.query(isLargerThan)

for row in qres_construct:
    print(row)

(rdflib.term.URIRef('mcs:object:parking_garage'), rdflib.term.URIRef('http://purl.org/twc/mcs/isLargerThan'), rdflib.term.URIRef('mcs:object:vehicle'))


In [17]:
qres_ask = g.query("ASK {mcs:object:parking_garage mcs:isLargerThan mcs:object:automobile}")
print(bool(qres_ask))

False
